In [ ]:

import numpy as np
from ikpy.chain import Chain
from ikpy.link import OriginLink, URDFLink
from dynamixel_sdk import PortHandler, PacketHandler

# Joint 7: Base joint, rotates about z direction
joint7 = URDFLink(
    name="7_base_rotation",
    origin_translation=np.array([0, 0, 0.038]),  # 38 mm upward from base
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, 0, 1]),
    joint_type="revolute",
    bounds=(0, 3*np.pi/2)
)

# Joint 8: Shoulder tilt (rotates about y, here using [0, -1, 0])
joint8 = URDFLink(
    name="8_shoulder_tilt",
    origin_translation=np.array([0, 0, 0.017]),  # 17 mm upward from Joint 7
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, 1, 0]),
    joint_type="revolute",
    bounds=(-.65, np.pi/2)
)

# Joint 9: Elbow tilt (rotates about y)
L2 = 0.110  # 110 mm from Joint 8
joint9 = URDFLink(
    name="9_elbow_tilt",
    origin_translation=np.array([0, 0, L2]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, 1, 0]),
    joint_type="revolute",
    bounds=(-110 / 180 * np.pi, 1.4)
)

# Joint 10: Wrist tilt (rotates about y)
L3 = 0.100  # 100 mm from Joint 9
joint10 = URDFLink(
    name="10_wrist_tilt",
    origin_translation=np.array([L3, 0, 0]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, -1, 0]),
    joint_type="revolute",
    bounds=(-np.pi/2, np.pi/2)
)

# Joint 11: Gripper rotate (rotates about z)
L4 = 0.120 # 45 mm from Joint 10
joint11 = URDFLink(
    name="11_gripper_rotate",
    origin_translation=np.array([0, 0, -L4]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([1, 0, 0]),
    joint_type="revolute"
)

# Assemble the robot chain: The first link is the fixed base (OriginLink)
robot_chain_follower = Chain(name="robot_chain_follower", links=[
    OriginLink(),
    joint7,
    joint8,
    joint9,
    joint10,
    joint11
])


PROTOCOL_VERSION = 2.0
DXL_IDS = [7, 8, 9, 10, 11, 12] 
BAUDRATE = 1e6
DEVICENAME = '/dev/tty.usbmodem58FD0170871'  # Replace with your serial port

TICKS_PER_REV = 4096

def rad_to_dxl(angle_rad):
    angle_rad = angle_rad
    return int((angle_rad / (2 * np.pi)) * TICKS_PER_REV)

def dxl_to_rad(ticks):
    return (ticks / TICKS_PER_REV) * 2 * np.pi

port_handler = PortHandler(DEVICENAME)
packet_handler = PacketHandler(PROTOCOL_VERSION)
if not port_handler.openPort():
    print("Failed to open port!")
    exit()
if not port_handler.setBaudRate(BAUDRATE):
    print("Failed to set baudrate!")
    exit()

def to_signed(val):
    if val > 0x7FFFFFFF:  # 0x7FFFFFFF is 2^31 - 1
        return val - 0x100000000  # 0x100000000 is 2^32
    return val


def read_current_angles_follower():
    angles = []
    for dxl_id in DXL_IDS[:5]:  # Only for joints 7-11
        pos, dxl_comm_result, dxl_error = packet_handler.read4ByteTxRx(port_handler, dxl_id, 132)
        # print("ID", dxl_id, "TICKS", pos)
        if dxl_comm_result != 0 or dxl_error != 0:
            print(f"Error reading servo {dxl_id}, assuming 0.")
            angles.append(0)
        else:
            angle = dxl_to_rad(to_signed(pos))
            # print("RAW POS", pos)
            # print("SIGNED POS", to_signed(pos))
            angles.append(angle)
    return np.array(angles)


def forward_kinematics_follower(angles):
    truncated = angles[:5]  # angles for joints 7-11
    full_angles = np.concatenate(([0], truncated))  # Prepend dummy for OriginLink
    return robot_chain_follower.forward_kinematics(full_angles)

current_angles = read_current_angles_follower()
print("Current joint angles (deg) from servos:", current_angles*180/ (np.pi))
print("Current joint angles (rad) from servos:", current_angles)

home_pose = forward_kinematics_follower(current_angles)
home_xyz = home_pose[:3, 3] * 100  # Convert from meters to centimeters

print("\nHome position (xyz) of end-effector (joint 11) relative to base (in cm):")
print(f"  x = {home_xyz[0]:.2f} cm")
print(f"  y = {home_xyz[1]:.2f} cm")
print(f"  z = {home_xyz[2]:.2f} cm")

port_handler.closePort()


Current joint angles (deg) from servos: [ -7.64648437  55.10742188 -59.58984375  20.7421875   82.96875   ]
Current joint angles (rad) from servos: [-0.13345633  0.96180595 -1.04003897  0.36201947  1.44807786]

Home position (xyz) of end-effector (joint 11) relative to base (in cm):
  x = 23.89 cm
  y = -3.21 cm
  z = 1.72 cm
